In [1]:
from jupyter_dash import JupyterDash
from dash import dcc
from dash import html
from dash.dependencies import Output, Input
import plotly.graph_objs as go

class TouchPlotter(object):
    def __init__(self, mode='inline', port=8050, width='100%', height='650px', view='position', max_x=0, max_y=0, plot_scale=1):
        self.mode = mode
        self.port = port
        self.width = width
        self.height = height
        self.pos = []
        self.trace =  [[[] for i in range(10)] for j in range(2)]
        self.trace_status = ['*'] * 10
        self.terminate = False
        self.clicks = 0
        self.params = {
            'maxX': max_x,
            'maxY': max_y,
            'view': view,
            'plotScale': plot_scale,
            'error': ''
        }
        self.plot = {
            'data': [go.Scatter(
                x = None,
                y = None
            )]
        }
        self.app = JupyterDash()
        self.app.layout = html.Div([
            dcc.Graph(
                id = 'touch-graph',
                figure = self.plot,
                config={'displayModeBar': False}
            ),
            html.Div(
                children = [
                    html.Button(
                        'Clear Traces',
                        id ='clear-button',
                        n_clicks = 0
                    )
                ] if (view == 'trace') else [
                    html.Button(
                        'Clear Traces',
                        id ='clear-button',
                        style = {'display': 'none'},
                        n_clicks = 0
                    )
                ]
            ),
            dcc.Store(
                id = 'pos-store',
                data = self.pos
            ),
            dcc.Store(
                id = 'trace-store',
                data = self.trace
            ),
            dcc.Store(
                id = 'params-store',
                data = self.params
            ),
            dcc.Interval(
                id = 'update-store',
                interval = 100,
                n_intervals = 0
            )
        ])
        self.app.clientside_callback(
            """
            function(pos, trace, params) {
              const errorOutput = {
                layout: {
                  title: {
                    font: {color: 'red'},
                    xref: 'x',
                    x: 0,
                  },
                  width: 1000,
                  height: 100,
                  xaxis: {visible: false},
                  yaxis: {visible: false}
                }
              };

              if (params.error !== '') {
                const errorMessage = params.error;
                errorOutput.layout.title.text = errorMessage;
                return errorOutput;
              }

              if (params.view === 'position' && !pos) {
                const errorMessage = 'No valid touch position data found';
                errorOutput.layout.title.text = errorMessage;
                return errorOutput;
              }

              if (params.view === 'trace' && !trace) {
                const errorMessage = 'No valid touch trace data found';
                errorOutput.layout.title.text = errorMessage;
                return errorOutput;
              }

              if (params.maxX === 0 || params.maxY === 0) {
                const errorMessage = 'Maximum X and Y values not set';
                errorOutput.layout.title.text = errorMessage;
                return errorOutput;
              }

              const MIN_HEIGHT = 300;
              const NRM_HEIGHT = 500;
              const MAX_HEIGHT = 700;

              const font_color = '#777777';
              const plot_bgcolor = 'black';
              const paper_bgcolor = 'rgba(0, 0, 0, 0)';

              const viridisColors = [
                '#440154',
                '#482878',
                '#3E4A89',
                '#31688E',
                '#26828E',
                '#1F9E89',
                '#35B779',
                '#6ECE58',
                '#B5DE2B',
                '#FDE725'
              ];

              const l = 2;
              const r = 110;
              const t = 2;
              const b = 10;

              let height = Math.floor(NRM_HEIGHT * params.plotScale);
              if (height < MIN_HEIGHT) {
                height = MIN_HEIGHT;
              } else if (height > MAX_HEIGHT) {
                height = MAX_HEIGHT;
              }
              let width = Math.floor(height * ((params.maxX + 1) / (params.maxY + 1)));

              width += (l + r);
              height += (t + b);

              const markers = [];
              const markerSize = Math.floor(height / 15);

              if (params.view === 'position') {
                const x = [...Array(10)].map(e => Array(1));
                const y = [...Array(10)].map(e => Array(1));

                for (let i = 0; i < pos.length; i++) {
                  const obj = pos[i];
                  const index = obj.objectIndex;
                  x[index][0] = obj.xMeas;
                  y[index][0] = obj.yMeas;
                }

                const textSize = Math.floor(markerSize / 2);

                for (let i = 0; i < 10; i++) {
                  const marker = {
                    x: x[i],
                    y: y[i],
                    mode: 'markers+text',
                    marker: {size: markerSize, color: viridisColors[i]},
                    text: [i.toString()],
                    textposition: 'inside',
                    textfont: {family: 'Arial', color: 'white', size: textSize},
                    name: 'Finger ' + i
                  };
                  if (i >= 5) {
                    marker.textfont.color = 'black';
                  }
                  markers.push(marker);
                }
              } else if (params.view === 'trace') {
                const traceWidth = Math.floor(markerSize / 8);
                for (let i = 0; i < 10; i++) {
                  const marker = {
                    x: trace[0][i],
                    y: trace[1][i],
                    mode: 'lines',
                    line: {shape: 'linear', width: traceWidth, color: viridisColors[i]},
                    name: 'Finger ' + i
                  };
                  markers.push(marker);
                }

                const dummyX = [];
                const dummyY = [];
                markers.push({x: dummyX, y: dummyY});
              }

              const figure = {
                data: markers,
                layout: {
                  width,
                  height,
                  margin: {l, r, t, b},
                  plot_bgcolor,
                  paper_bgcolor,
                  font: {
                    color: font_color
                  },
                  xaxis: {
                    range: [0, params.maxX],
                    mirror: true,
                    showline: true,
                    linecolor: '#A9A9A9',
                    showticklabels: false
                  },
                  yaxis: {
                    range: [0, params.maxY],
                    mirror: true,
                    showline: true,
                    linecolor: '#A9A9A9',
                    showticklabels: false
                  },
                  showlegend: params.view === 'trace'
                }
              };

              return figure;
            }
            """,
            Output('touch-graph', 'figure'),
            Input('pos-store', 'data'),
            Input('trace-store', 'data'),
            Input('params-store', 'data')
        )
        self.app.callback(
            [Output('pos-store', 'data'),
            Output('trace-store', 'data'),
            Output('params-store', 'data'),
            Output('update-store', 'disabled')],
            [Input('update-store', 'n_intervals'),
            Input('clear-button', 'n_clicks')]
        )(self.__update_stores)

    def __update_stores(self, n_intervals, n_clicks):
        if (n_clicks != self.clicks):
            self.clicks = n_clicks
            self.trace =  [[[] for i in range(10)] for j in range(2)]
            self.trace_status = ['*'] * 10
        return [self.pos, self.trace, self.params, self.terminate]

    def __capture_traces(self):
        for i in range(10):
            if (self.trace_status[i] == '+'):
                self.trace_status[i] = '-'

        for i in range(len(self.pos)):
            obj = self.pos[i]
            index = int(obj['objectIndex'])
            if (self.trace_status[index] == '*'):
                self.trace[0][index] = [obj['xMeas']]
                self.trace[1][index] = [obj['yMeas']]
            else:
              self.trace[0][index].append(obj['xMeas'])
              self.trace[1][index].append(obj['yMeas'])
            self.trace_status[index] = '+'

        for i in range(10):
            if (self.trace_status[i] == '-'):
                self.trace_status[i] = '*'

    def run_plot(self):
        self.app.run_server(mode = self.mode, host = 'localhost', port = self.port, width = self.width, height = self.height)

    def update_plot(self, report):
        if ('position' in report):
            if ('pos' in report[1]):
                self.pos = report[1]['pos']
            else:
                self.pos = []
            self.__capture_traces()

    def show_error(self, error):
        self.params['error'] = error

    def stop_plot(self):
        self.terminate = True
